<a href="https://colab.research.google.com/github/chenkaiwen111811/PL-Repo/blob/main/%E4%BD%9C%E6%A5%AD%E4%BA%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install google-search-results folium geopy -q

In [33]:
from serpapi import GoogleSearch
import folium
from geopy.geocoders import Nominatim
import json
import re
from datetime import datetime
from IPython.display import display
import gradio as gr

In [34]:
SERPAPI_KEY = "eb99ac7f29108fa47d1feadfc2613b63706fd17bbf6919c5da26e340e15111f2"

In [35]:
# 初始化地理編碼器
geolocator = Nominatim(user_agent="search_map_app")


TAIWAN_LOCATIONS = [
    # 直轄市
    '台北', '台北市', '臺北', '臺北市', 'Taipei',
    '新北', '新北市', 'New Taipei',
    '桃園', '桃園市', 'Taoyuan',
    '台中', '台中市', '臺中', '臺中市', 'Taichung',
    '台南', '台南市', '臺南', '臺南市', 'Tainan',
    '高雄', '高雄市', 'Kaohsiung',

    # 縣市
    '基隆', '基隆市', 'Keelung',
    '新竹', '新竹市', '新竹縣', 'Hsinchu',
    '苗栗', '苗栗縣', 'Miaoli',
    '彰化', '彰化縣', 'Changhua',
    '南投', '南投縣', 'Nantou',
    '雲林', '雲林縣', 'Yunlin',
    '嘉義', '嘉義市', '嘉義縣', 'Chiayi',
    '屏東', '屏東縣', 'Pingtung',
    '宜蘭', '宜蘭縣', 'Yilan',
    '花蓮', '花蓮縣', 'Hualien',
    '台東', '台東縣', '臺東', '臺東縣', 'Taitung',
    '澎湖', '澎湖縣', 'Penghu',
    '金門', '金門縣', 'Kinmen',
    '連江', '連江縣', '馬祖', 'Matsu',

    # 知名景點/地標 (原有的)
    '101', '台北101', '西門町', '信義區', '中正區',
    '士林', '北投', '淡水', '九份', '平溪',
    '日月潭', '阿里山', '墾丁', '太魯閣',
    '東大門', '逢甲', '一中街', '勤美',
    '大安區', '中山區', '松山區', '萬華區', '內湖區',
    '板橋', '永和', '中和',
    '中壢', '藝文特區',
    '西屯區', '南屯區', '北屯區', '西區', '中區',
    '東區', '中西區', '安平區',
    '鼓山區', '左營區', '三民區', '新興區', '鹽埕區',
    '赤峰街', '永康街', '公館', '天母', '民生社區',
    '審計新村', '駁二', '國父紀念館', '松菸', '華山'
]


def search_google(query, api_key, num=5, gl='tw', hl='zh-tw'):
    """
    使用 SerpAPI 進行 Google 搜尋
    """
    try:
        params = {
            'q': query,
            'api_key': api_key,
            'num': num,
            'gl': gl,
            'hl': hl
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        return results.get('organic_results', [])

    except Exception as e:
        print(f"搜尋時發生錯誤: {e}")
        return []

def extract_locations_from_text(text):
    """
    從文字中擷取地區名稱

    參數:
        text (str): 要分析的文字

    回傳:
        list: 找到的地區名稱列表
    """
    found_locations = []
    text_lower = text.lower()

    for location in TAIWAN_LOCATIONS:
        if location.lower() in text_lower:
            # 避免重複加入相似地名
            base_name = location.replace('市', '').replace('縣', '')
            if base_name not in [loc.replace('市', '').replace('縣', '') for loc in found_locations]:
                found_locations.append(location)

    return found_locations


def get_coordinates(location_name):
    """
    取得地點的經緯度座標

    參數:
        location_name (str): 地點名稱

    回傳:
        tuple: (緯度, 經度) 或 None
    """
    try:
        # 加上台灣以提高準確度
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except Exception as e:
        print(f"無法取得 {location_name} 的座標: {e}")
        return None


def analyze_results_with_locations(results):
    """
    分析搜尋結果並擷取地理位置資訊

    參數:
        results (list): 搜尋結果列表

    回傳:
        list: 包含位置資訊的結果字典列表
    """
    analyzed_results = []

    for result in results:
        # 從標題和描述中尋找地點
        title = result.get('title', '')
        snippet = result.get('snippet', '')
        combined_text = f"{title} {snippet}"

        locations = extract_locations_from_text(combined_text)

        if locations:
            analyzed_results.append({
                'title': title,
                'snippet': snippet,
                'link': result.get('link', ''),
                'locations': locations
            })

    return analyzed_results


def create_map_from_results(analyzed_results, center=[23.5, 121.0], zoom=7):
    """
    根據分析結果建立地圖

    參數:
        analyzed_results (list): 包含位置資訊的結果列表
        center (list): 地圖中心座標 [緯度, 經度]
        zoom (int): 地圖縮放層級

    回傳:
        folium.Map: 地圖物件
    """
    # 建立地圖
    m = folium.Map(location=center, zoom_start=zoom, tiles='OpenStreetMap')

    # 用於記錄已標註的位置，避免重複
    marked_locations = {}

    # 為每個結果加上標記
    for result in analyzed_results:
        for location in result['locations']:
            coords = get_coordinates(location)

            if coords:
                lat, lon = coords

                # 如果該位置還沒標記過
                if location not in marked_locations:
                    marked_locations[location] = []

                # 將結果加到該位置
                marked_locations[location].append(result)

    # 在地圖上標註
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
              'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue']

    for i, (location, results) in enumerate(marked_locations.items()):
        coords = get_coordinates(location)
        if coords:
            lat, lon = coords

            # 建立 popup 內容
            popup_html = f"<div style='width: 300px;'>"
            popup_html += f"<h4 style='color: #2c3e50;'>{location}</h4>"
            popup_html += f"<p style='color: #7f8c8d; font-size: 12px;'>找到 {len(results)} 筆相關結果</p>"

            for j, res in enumerate(results, 1):
                popup_html += f"<div style='margin: 10px 0; padding: 10px; background: #ecf0f1; border-radius: 5px;'>"
                popup_html += f"<b style='color: #2980b9;'>[{j}] {res['title'][:50]}...</b><br>"
                popup_html += f"<small style='color: #34495e;'>{res['snippet'][:100]}...</small><br>"
                popup_html += f"<a href='{res['link']}' target='_blank' style='color: #3498db;'>查看連結</a>"
                popup_html += "</div>"

            popup_html += "</div>"

            # 加上標記
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_html, max_width=350),
                tooltip=f"{location} ({len(results)} 筆結果)",
                icon=folium.Icon(color=colors[i % len(colors)], icon='info-sign')
            ).add_to(m)

    return m


def generate_results_summary_string(analyzed_results):
    """
    產生分析結果摘要的 "字串"
    """
    summary_str = "" # 改用字串累加
    summary_str += "="*80 + "\n"
    summary_str += f"找到 {len(analyzed_results)} 筆包含地理位置的搜尋結果\n"
    summary_str += "="*80 + "\n\n"

    for i, result in enumerate(analyzed_results, 1):
        summary_str += f"【結果 {i}】\n"
        summary_str += f"標題: {result['title']}\n"
        summary_str += f"地點: {', '.join(result['locations'])}\n"
        summary_str += f"描述: {result['snippet'][:100]}...\n"
        summary_str += f"網址: {result['link']}\n"
        summary_str += "-"*80 + "\n\n"
    return summary_str

In [ ]:
def gradio_search_and_map(location_query, style_query, num_results_slider):

    # --- 1. 組合輸入 (來自你的邏輯) ---
    num_results = int(num_results_slider) # Gradio slider 可能是 float
    search_term = "咖啡廳"

    if style_query:
        custom_query = f"{location_query} {style_query} {search_term}"
    else:
        custom_query = f"{location_query} {search_term}"

    # 這是給摘要框的「過程」更新
    status_update = f"正在搜尋: 「{custom_query}」，最多 {num_results} 筆...\n"

    # --- 2. 呼叫你的 'search_google' 函式 ---
    results = search_google(custom_query, SERPAPI_KEY, num=num_results)
    if not results:
        # 回傳 (摘要, 地圖)
        return status_update + "沒有找到任何結果", None

    status_update += f"✓ 找到 {len(results)} 筆搜尋結果\n"

    # --- 3. 呼叫你的 'analyze_results_with_locations' 函式 ---
    analyzed_results = analyze_results_with_locations(results)
    if not analyzed_results:
        no_location_msg = f"\n⚠️ 搜尋結果中沒有找到 {TAIWAN_LOCATIONS} 列表中的可辨識地區"
        return status_update + no_location_msg, None

    status_update += f"✓ 從結果中找到 {len(analyzed_results)} 筆包含地理位置的資料\n"

    # --- 4. 產生摘要 "字串" (使用新的輔助函式) ---
    summary_text = status_update + generate_results_summary_string(analyzed_results)

    # --- 5. 呼叫你的 'create_map_from_results' 函式 ---
    print("正在建立地圖...\n")
    map_obj = create_map_from_results(analyzed_results)
    print("✓ 地圖建立完成！")

    # --- 6. 準備 Gradio 輸出 ---
    if map_obj:
        # 儲存地圖 (保留你的原始邏輯)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"cafe_map_{timestamp}.html"
        map_obj.save(filename)
        summary_text += f"\n✓ 地圖已儲存為: {filename}"

        # 將 Folium 地圖轉為 HTML 字串
        map_html = map_obj._repr_html_()

        # 回傳 (摘要字串, 地圖HTML)
        return summary_text, map_html
    else:
        return summary_text + "\n❌ 地圖建立失敗", None

# 8. --- NEW: 啟動 Gradio 介面 ---
# 我們使用 gr.Blocks 來自訂排版
with gr.Blocks(title="咖啡廳地圖搜尋器") as demo:
    gr.Markdown("# ☕ 咖啡廳地圖搜尋器")
    gr.Markdown("輸入地點和偏好，自動在 Google 搜尋並標記在地圖上。")

    with gr.Row():
        with gr.Column(scale=1):
            # --- 輸入元件 ---
            location_input = gr.Textbox(label="📍 地點", placeholder="例如: 台北車站, 台中勤美, 大安區")
            style_input = gr.Textbox(label="✨ 風格/特色", placeholder="例如: 不限時, 安靜, 寵物友善 (可選)")
            num_slider = gr.Slider(minimum=5, maximum=20, value=10, step=1, label="🔍 搜尋結果數量")
            submit_btn = gr.Button("開始搜尋", variant="primary")

        with gr.Column(scale=2):
            # --- 輸出元件 1: 摘要 ---
            summary_output = gr.Textbox(
                label="📊 搜尋結果摘要",
                lines=15,
                interactive=False,
                show_copy_button=True
            )

    # --- 輸出元件 2: 地圖 ---
    map_output = gr.HTML(label="🗺️ 互動地圖")

    # --- 設定按鈕點擊事件 ---
    submit_btn.click(
        fn=gradio_search_and_map,  # 按下按鈕時，呼叫這個函式
        inputs=[location_input, style_input, num_slider], # 接收這三個元件的
        outputs=[summary_output, map_output] # 並將結果回傳給這兩個元件
    )

# 執行 App (在 Colab/Jupyter 中會顯示)
# share=True 會產生一個公開的臨時網址
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://93c183007d9b13504c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


正在建立地圖...

✓ 地圖建立完成！
